---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re
import io
pd.set_option('display.max_rows', 520)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [11]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [12]:
def strip_string(s):    
    s = re.sub(r"\s*\[.*\s*","", s)
    s = re.sub(r"\s*\(.*\s*","", s)
    s = re.sub(u"\u2014", "", s)
    s = s.strip()
    return(s)

In [69]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    v = []
    state = ""
    region = ""        
    #lines = open('university_towns.txt').read().splitlines()
    lines = io.open('university_towns.txt', mode = "r", encoding = "utf-8").read().splitlines()
    df = pd.DataFrame(lines, columns = list("T"))
    df['State'] = df["T"].apply(lambda x: x.split('[edit]')[0].strip() if x.count('[edit]') > 0 else np.NaN).fillna(method="ffill")
    df['RegionName'] = df["T"].apply(lambda x: strip_string(x) if x.count("[edit]") == 0  else np.NaN)
    df = df.dropna().drop('T', axis=1).reindex(columns=['State', 'RegionName']).reset_index(drop=True)
    return (df)

In [70]:
def get_gdp():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    cols = ["Quarter", "GDP"]
    gdp = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = cols) 
    gdp["GDP-1"] = gdp["GDP"].shift(1)
    gdp["GDP-2"] = gdp["GDP"].shift(2)
    gdp["GDP+1"] = gdp["GDP"].shift(-1)
    gdp["GDP+2"] = gdp["GDP"].shift(-2)
    return gdp
#get_gdp()

In [71]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    gdp["RecessionStart"] = (gdp["GDP-1"] > gdp["GDP"]) & (gdp["GDP"] > gdp["GDP+1"]) &(gdp["GDP+1"] > gdp["GDP+2"])
    qtr = gdp[gdp["RecessionStart"] == True]
    return qtr["Quarter"].min()
#get_recession_start()

In [76]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    start = get_recession_start()
    gdp["RecessionEnd"] = (gdp["GDP"] > gdp["GDP-1"]) & (gdp["GDP-1"] > gdp["GDP-2"]) & (gdp["Quarter"] > start)
    qtr = gdp[gdp["RecessionEnd"] == True]
    return qtr["Quarter"].min()
#get_recession_end()

'2009q4'

In [73]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    start = get_recession_start()
    end = get_recession_end()
    recession = gdp[(gdp["Quarter"] >= start) & (gdp["Quarter"] <= end)]
    bottom = recession.loc[recession["GDP"].idxmin(), "Quarter"]    
    return bottom

In [74]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Read housing data
    housing = pd.read_csv("City_Zhvi_AllHomes.csv", usecols = list(range(1,3)) + list(range(51, 251)))
    housing["State"] = housing["State"].map(states)
    housing = housing.set_index(["State", "RegionName"])
        
    # Separate time columns
    tdf = housing[housing.columns[1:]].rename(columns=pd.to_datetime)
    tcols = tdf.columns
    
    # Select columns, resample, rename
    qdf = tdf[tcols].resample('Q',axis=1).mean().rename(
    columns=lambda x: str(x.to_period("Q")).lower())
    
    return qdf

In [75]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Obtain data    
    hdf = convert_housing_data_to_quarters()
    start = get_recession_start()
    bottom = get_recession_bottom()
    ut = get_list_of_university_towns()
    
    # Set quarter before start of recession
    qb4 = hdf.columns[hdf.columns.get_loc(start)-1]
    
    # Add price ratio column
    hdf["PriceRatio"] = hdf[qb4].div(hdf[bottom])
    
    # Split data into university and non-university town data
    ut_list = ut.to_records(index = False).tolist()
    ut_hdf = hdf.loc[ut_list]
    non_ut_hdf = hdf.loc[~(hdf.index.isin(ut_list))]
    
    # Conduct t-test
    t, p = ttest_ind(ut_hdf["PriceRatio"], non_ut_hdf["PriceRatio"], equal_var = True, nan_policy = "omit")
    #print('ttest_ind result: t={}, pvalue={}'.format(t, p))
    
    # Format output
    if p < .01:
        different = True
    else:
        different = False
    
    if (t > 0):
        better = "non-university town"
    else:
        better = "university town"

    res = (different, p, better)
    return res